# Running a Simple R Script in Bacalhau

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bacalhau-project/examples/blob/main/workload-onboarding/r-hello-world/index.ipynb)
[![Open In Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/bacalhau-project/examples/HEAD?labpath=workload-onboarding/r-hello-world/index.ipynb)
[![stars - badge-generator](https://img.shields.io/github/stars/bacalhau-project/bacalhau?style=social)](https://github.com/bacalhau-project/bacalhau)

You can use official Docker containers for each language like R or Python. In this example, we will use the official R container and run it on bacalhau. 

## TD;LR
A quick guide on how to run a hello world script on Bacalhau

## Prerequisites

To get started, you need to install the Bacalhau client, see more information [here](https://docs.bacalhau.org/getting-started/installation)

In [ ]:
!command -v bacalhau >/dev/null 2>&1 || (export BACALHAU_INSTALL_DIR=.; curl -sL https://get.bacalhau.org/install.sh | bash)
path=!echo $PATH
%env PATH=./:{path[0]}

env: PATH=./:/Users/phil/.cargo/bin:/Users/phil/.pyenv/versions/3.9.7/bin:/opt/homebrew/Caskroom/google-cloud-sdk/latest/google-cloud-sdk/bin:/Users/phil/.gvm/bin:/opt/homebrew/opt/findutils/libexec/gnubin:/opt/homebrew/opt/coreutils/libexec/gnubin:/opt/homebrew/Caskroom/google-cloud-sdk/latest/google-cloud-sdk/bin:/Users/phil/.pyenv/shims:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/usr/local/MacGPG2/bin:/Users/phil/.nexustools


## Running an R Script Locally

To install R follow these instructions [A Installing R and RStudio | Hands-On Programming with R](https://rstudio-education.github.io/hopr/starting.html). After R and RStudio is installed, create and run a script called hello.R.

In [ ]:
%%writefile hello.R
print("hello world")

Run the script:

In [ ]:
%%bash
Rscript hello.R

Next, upload the script to your public storage in our case IPFS.  We've already uploaded the script to IPFS and the CID is: `QmVHSWhAL7fNkRiHfoEJGeMYjaYZUsKHvix7L54SptR8ie`. You can look at this by browsing to one of the HTTP IPFS proxies like [ipfs.io](https://cloudflare-ipfs.com/ipfs/QmVHSWhAL7fNkRiHfoEJGeMYjaYZUsKHvix7L54SptR8ie/) or [w3s.link](https://w3s.link/ipfs/QmVHSWhAL7fNkRiHfoEJGeMYjaYZUsKHvix7L54SptR8ie).

## Running a Job on Bacalhau

Now it's time to run the script on the Bacalhau network. To run a job on Bacalhau, run the following command:

In [2]:
%%bash --out job_id
bacalhau docker run \
--wait \
--id-only \
-v QmQRVx3gXVLaRXywgwo8GCTQ63fHqWV88FiwEqCidmUGhk:/hello.R \
r-base \
-- Rscript hello.R

### Structure of the command

Let's look closely at the command above:

* `bacalhau docker run`: call to bacalhau 
  
* `-v QmQRVx3gXVLaRXywgwo8GCTQ63fHqWV88FiwEqCidmUGhk`: CIDs to use on the job. Mounts them at '/inputs' in the execution.

* `:/hello.R`: the name and the tag of the docker image we are using

* `Rscript hello.R`: execute the R script


When a job is submitted, Bacalhau prints out the related `job_id`. We store that in an environment variable so that we can reuse it later on.

In [ ]:
%env JOB_ID={job_id}


## Checking the State of your Jobs

- **Job status**: You can check the status of the job using `bacalhau list`.


In [ ]:
%%bash
bacalhau list --id-filter ${JOB_ID}

When it says `Published` or `Completed`, that means the job is done, and we can get the results.

- **Job information**: You can find out more information about your job by using `bacalhau describe`.

In [ ]:
%%bash
bacalhau describe  ${JOB_ID}

- **Job download**: You can download your job results directly by using `bacalhau get`. Alternatively, you can choose to create a directory to store your results. In the command below, we created a directory and downloaded our job output to be stored in that directory.

In [6]:
%%bash
rm -rf results && mkdir results
bacalhau get ${JOB_ID} --output-dir results

## Viewing your Job Output

Each job creates 3 subfolders: the **combined_results**, **per_shard files**, and the **raw** directory. To view the file, run the following command:

In [ ]:
%%bash
ls results/

Viewing the result

In [9]:
%%bash
cat results/combined_results/stdout

[1] "hello world"


### Futureproofing your R Scripts

You can generate the the job request with the following command. This will allow you to re-run that job in the future.

In [10]:
%%bash
bacalhau describe ${JOB_ID} --spec > job.yaml

In [ ]:
%%bash
cat job.yaml